In [106]:
class Car:
    """
    As a convention, all horizontally aligned vehicles face right and all vertically aligned vehicles face down.
    This convention makes no difference to the algorithm, but makes easier to write the code.
    """
    def __init__(self, pos, size, name):
        """
        @input:
            pos: a list of two lists
                 it contains two lists of length two which are the positions of the vehicle's ends

            size: the size of the grid
            name: a letter indicating the name of the car
        """
        self.size = size
        self.name = name
        self.f_r = pos[0][0]
        self.f_c = pos[0][1]
        self.b_r = pos[1][0]
        self.b_c = pos[1][1]
        if self.f_r == self.b_r: # aligned horizontally
            self.alignment = 'H'
            if self.f_c < self.b_c: # vehicle is faacing left, swap
                tmp = self.f_c
                self.f_c = self.b_c
                self.b_c = tmp
            self.length = self.f_c - self.b_c + 1
        else:
            self.alignment = 'V'
            if self.f_r < self.b_r: # vehicle is facing up, swap
                tmp = self.f_r
                self.f_r = self.b_r
                self.b_r = tmp
            self.length = self.f_r - self.b_r + 1
        
    def to_str(self):
        """
        The to_str method has to be explicitly called becuase we want to be able to store references to the graph
        objects in multiple places; thus not overwriting __repr__
        """
        return "[Pos: [{}, {}], [{}, {}], Name: {}, Alignment: {}, Length: {}]".format(self.f_r, self.f_c, 
                                                                                       self.b_r, self.b_c,
                                                                                       self.name, self.alignment,
                                                                                      self.length)
    
    def __eq__(self, car):
        return self.to_str() == car.to_str()
    
    def is_blocking(self, cell):
        """
        Given a cell, is this car occupying that cell
        If the cell is in the same column as the (vertically aligned) car, then check if the cell's row is within 
        the two rows that mark the ends of the car.
        If the cell is in the same row as the (horizontally aligned) car, then the same check as above is done on the
        columns.
        """
        if self.alignment == 'V':
            if self.f_c == cell[1]: # is cell in the same column as the car
                return self.b_r <= cell[0] <= self.f_r
        if self.alignment == 'H':
            if self.f_r == cell[0]: # is cell in the same row as the car
                return self.b_c <= cell[1] <= self.f_c
        return False # the cell is not in the same column or the same row as the car
 
    def move(self, d):
        """
        Move the vehicle one cell in d direction
        @input:
            d: direction to move - 'U' or 'D' when alignment is vertical or 'L' or 'R' otherwise
        """
        if self.alignment == 'V':
            if d == 'U':
                if self.b_r > 0: # upward move is valid
                    self.b_r -= 1
                    self.f_r -= 1
            if d == 'D':
                if self.f_r < size - 2: # downward move is valid
                    self.b_r += 1
                    self.f_r += 1
        if self.alignment == 'E':
            if d == 'L':
                if self.b_c > 0: # leftward move is valid
                    self.b_c -= 1
                    self.f_c -= 1
            if d == 'R':
                if self.f_c < size - 2: # rightward move is valid
                    self.b_c += 1
                    self.f_c += 1

In [11]:
def get_neighborhood(car, cell):
    """
    The cell needs to be unblocked. The neighborhood is with respect to the cell
    """
    # TODO: Should this function be moved into the the Car class?
    assert isinstance(car, Car)
    rng = car.n_range(cell)
    if car.alignment == 'V':
        n = {'U': [], 'D': []}
        for i in range(1, rng[0] + 1): # cells upward
            n['U'].append([car.b_r - i, car.b_c])
        for i in range(1, rng[1] + 1):
            n['D'].append([car.f_r + i, car.f_c])
    elif car.alignment == 'H':
        n = {'L': [], 'R': []}
        for i in range(1, rng[0] + 1):
            n['L'].append([car.b_r, car.b_c - i])
        for i in range(1, rng[1] + 1):
            n['R'].append([car.f_r, car.f_c + i])
    return n

In [10]:
# Initialize all cars
size = 6
cars = []
cars.append(Car([[2, 0], [2, 1]], size, 'R'))
cars.append(Car([[0, 1], [1, 1]], size, 'S'))
cars.append(Car([[0, 2], [0, 3]], size, 'F'))
cars.append(Car([[0, 4], [0, 5]], size, 'G'))
cars.append(Car([[1, 4], [1, 5]], size, 'H'))
cars.append(Car([[1, 3], [2, 3]], size, 'I'))
cars.append(Car([[3, 0], [5, 0]], size, 'C'))
cars.append(Car([[3, 1], [3, 3]], size, 'B'))
cars.append(Car([[4, 2], [5, 2]], size, 'K'))
cars.append(Car([[2, 4], [4, 4]], size, 'A'))
cars.append(Car([[2, 5], [3, 5]], size, 'M'))
cars.append(Car([[4, 5], [5, 5]], size, 'P'))
for car in cars:
    print(car.to_str())

[Pos: [2, 1], [2, 0], Name: R, Alignment: H, Length: 2]
[Pos: [1, 1], [0, 1], Name: S, Alignment: V, Length: 2]
[Pos: [0, 3], [0, 2], Name: F, Alignment: H, Length: 2]
[Pos: [0, 5], [0, 4], Name: G, Alignment: H, Length: 2]
[Pos: [1, 5], [1, 4], Name: H, Alignment: H, Length: 2]
[Pos: [2, 3], [1, 3], Name: I, Alignment: V, Length: 2]
[Pos: [5, 0], [3, 0], Name: C, Alignment: V, Length: 3]
[Pos: [3, 3], [3, 1], Name: B, Alignment: H, Length: 3]
[Pos: [5, 2], [4, 2], Name: K, Alignment: V, Length: 2]
[Pos: [4, 4], [2, 4], Name: A, Alignment: V, Length: 3]
[Pos: [3, 5], [2, 5], Name: M, Alignment: V, Length: 2]
[Pos: [5, 5], [4, 5], Name: P, Alignment: V, Length: 2]


In [105]:
class Node:
    """
    A node is the entire grid with all the cars at each step
    If two nodes have the cars at the same positions then they are the same node
    """
    def __init__(self, cars):
        """
        @input:
            cars: list of Car objects to initialise the grid
            size: grid is of dimension size x size
            exit: the goal cell
        """
        self.cars = cars
        self.get_red_car() # Mark the red car
        
        self.parent = None
        self.children = []() # there can be many children
    
    def __find_car__(self, car_name):
        for car in self.cars:
            if car.name == car_name:
                return car
    
    def __eq__(self, node):
        """
        redefine how nodes are compared
        
        this is a heavy comparison
        """
        res = True
        for self_car in self.cars:
            node_car = node.__find_car__(self_car.name)
            if self_car != node_car:
                res = False
        return res
        
    def node_copy_constructor(self):
        """
        This is a copy constructor to create a new node from this node
        This will create a new object in memory
        """
        return Node(self.cars)
    
    def get_red_car(self):
        for car in self.cars:
            if car.name == 'R':
                self.red = car
                break

In [107]:
class Graph:
    def __init__(self, cars, size, exit):
        self.start_node = Node(cars)
        self.size = size
        self.exit = exit
        self.current_node = self.start_node
        
        self.all_nodes = []() # holds all unique nodes created so far
        self.all_nodes.append(self.current_node)
        
        
        # red car always needs to go straight to reach goal cell
        self.path = [[self.current_node.red.f_r, self.current_node.red.f_c + i] \
                     for i in range(size - self.current_node.red.f_c)]
        # assume red car is never at goal cell
        self.explore_cell = [self.current_node.red.f_r, self.current_node.red.f_c + 1]
        
        # these nodes have been completely explored, we don't want to waste time on these anymore
        self.explored_nodes = []()
#         # maintain a list of created nodes, we don't want to create duplicate nodes
#         self.visited_nodes = []()
    
    def create_new_node(self):
        node = self.current_node.node_copy_constructor() # create a new node
        node.parent = self.current_node # parent node of node is current node
        self.current_node.children.append(node) # current node's child is node
        self.all_nodes.append(node)
    
    def blocking_car(self, cell):
        """
        Return the car that is blocking this cell
        """
        for car in self.current_node.cars:
            if car.is_blocking(cell):
                return car
        return None

    def has_reached_goal(self):
        return [self.current_node.red.f_r, self.current_node.red.f_c] == self.exit
        
    def can_car_move(self, car):
        n = []
        # car is vertically aligned
        if car.alignment == 'V':
            if car.b_r > 0:
                cell = [car.b_r - 1, car.b_c]
                if not self.blocking_car(cell):
                    n.append('U')
            if car.f_r < size - 2:
                cell = [car.f_r + 1, car.f_c]
                if not self.blocking_car(cell):
                    n.append('D')
        # car is horizontally aligned
        if car.alignment == 'H':
            if car.b_c > 0:
                cell = [car.b_r, car.b_c - 1]
                if not self.blocking_car(cell):
                    n.append('L')
            if car.f_c < size - 2:
                cell = [car.f_r, car.f_c + 1]
                if not self.blocking_car(cell):
                    n.append('R')
        return n
    
    def create_children(self):
        """
        This function looks at all the cars in the node
        and for every car, checks if the car can be moved in one or two directions
        If it can be moved, then one child node is created for every car and each direction in which the 
        car can move
        """
        for car in self.current_node.cars:
            n = self.can_car_move(car)
            if n:
                for d in n:
                    self.__move__(car.name, d)
        self.__verify_children__() # verify all children
        
        
    def is_current_node_explored(self):
        """
        The assumption is that all children of current node has already been exposed and goal not found
        this function backtracks to a parent node if children node list is empty
        """
        if len(self.current_node.children) == 0:
            self.explored_nodes.append(self.current_node)
            self.current_node = self.current_node.parent
            
    
    def __verify_children__(self):
        # for every node, check if we have seen this child before
        # if the child node is empty after running this function then we have completely explored 
        # self.current_node
        
        # again not part of the API, called only by "create_children"
        remove = []
        for i, node in enumerate(self.current_node.children):
            if node not in self.all_nodes:
                remove.append(i) # don't alter children while iterating
        
        for i in remove:
            self.current_node.children.pop(i) # remove nodes that we have seen earlier
        
        for node in self.current_node.children: # add all new and unique nodes to current node's children
            self.all_nodes.append(node)

    
    def __move__(self, car_name, d):
        # This function is used to generate a new node from the current node by moving one car along d.

        # This function performs no checks because it is assumed that this function is called only by
        # "create_children" and thus all verifications are done by "can_car_move"
        
        # This function is not part of the API and should not be called from outside the class
        
        if self.has_reached_goal():
            print("Goal has been reached")
            return
        
        self.create_new_node()
        node = self.current_node.children[-1]
        car = None
        
        # Assume car name is valid
        for c in node.cars:
            if c.name == car_name:
                car = c # Assumption - this is a pointer to the car object inside this newly created node object
                break
        
        if d == 'U':
            car.b_r -= 1
            car.f_r -= 1
        if d == 'D':
            car.b_r += 1
            car.f_r += 1
        if d == 'L':
            car.b_c -= 1
            car.f_c -= 1

In [74]:
EXIT = [2, 5]
graph = Graph(cars, size, EXIT)
graph.create_new_node()
for car in graph.current_node.cars:
    print(car.to_str())

[Pos: [2, 1], [2, 0], Name: R, Alignment: H, Length: 2]
[Pos: [1, 1], [0, 1], Name: S, Alignment: V, Length: 2]
[Pos: [0, 3], [0, 2], Name: F, Alignment: H, Length: 2]
[Pos: [0, 5], [0, 4], Name: G, Alignment: H, Length: 2]
[Pos: [1, 5], [1, 4], Name: H, Alignment: H, Length: 2]
[Pos: [2, 3], [1, 3], Name: I, Alignment: V, Length: 2]
[Pos: [5, 0], [3, 0], Name: C, Alignment: V, Length: 3]
[Pos: [3, 3], [3, 1], Name: B, Alignment: H, Length: 3]
[Pos: [5, 2], [4, 2], Name: K, Alignment: V, Length: 2]
[Pos: [4, 4], [2, 4], Name: A, Alignment: V, Length: 3]
[Pos: [3, 5], [2, 5], Name: M, Alignment: V, Length: 2]
[Pos: [5, 5], [4, 5], Name: P, Alignment: V, Length: 2]


In [82]:
cell = graph.explore_cell
[cell[0], cell[1] + 1] if cell[1] < size - 2 else []
cell

[2, 2]

In [102]:
d = []
for i in range(4, 10):
    d.append(i)

In [103]:
d.remove(7)

In [104]:
d

[4, 5, 6, 8, 9]